**Les bibliothèques:**

In [ ]:
!pip install rdflib
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install icalendar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 9.2 MB/s eta 0:00:00


In [ ]:
from rdflib import URIRef, BNode, Literal, Namespace
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO

In [ ]:
from icalendar import Calendar, Event, vCalAddress, vText 
from datetime import datetime
from pathlib import Path
import os
import pytz

**The first Container** (don't use this code)

In [ ]:
# init the calendar
from datetime import datetime, timedelta
from rdflib import Graph
import requests 
from base64 import b64encode
g1 = Graph()

schema = Namespace("http://schema.org/")
territoire = Namespace("https://territoire.emse.fr/kg/emse/fayol")
EX = Namespace('https://territoire.emse.fr/ldp/AmineAhmad3/#')

g1.bind('schema', SDO)
g1.bind('ex', EX)
g1.bind('emse',territoire)

name=EX['version']
g1.add((name,URIRef("http://www.w3.org/2000/01/rdf-schema#label"), Literal("SemWebProject", lang='en')))
g1.add((name,URIRef("http://www.w3.org/2000/01/rdf-schema#comment"), Literal("La liste des evenements", lang='fr')))
with open("version.ttl", "wb") as f:
    f.write(g1.serialize(format="ttl").encode())

In [ ]:
import requests 
from base64 import b64encode
# the authentication credentials


url = "https://territoire.emse.fr/ldp/"
slug = 'AmineAhmad'
username = 'ldpuser'
password = 'LinkedDataIsGreat'
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')
headers = {
        'Content-Type': 'text/turtle',
        'Authorization': f'Basic {encoded_credentials}',
        'Slug': slug,
    }

with open("/content/version.ttl", "r") as f:
    data = f.read()
    response = requests.post(url, headers=headers, data=data)

print("Status code:", response.status_code)

if response.status_code == 201:
    location = response.headers.get("Location")
    print("Created resource:", location)
else:
    print("Error occured :", response.content)

Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/


**EVENT : COURSE** (don't use this code)

In [ ]:
# init the calendar
from datetime import datetime, timedelta
from rdflib import Graph
import requests 
from base64 import b64encode
#url
url = "https://territoire.emse.fr/ldp/AmineAhmad3/"
#slug = 'AmineAhmad'
username = 'ldpuser'
password = 'LinkedDataIsGreat'
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')

headers = {
        'Content-Type': 'text/turtle',
        'Authorization': f'Basic {encoded_credentials}',
        'Slug': slug,
    }
# the authentication credentials
e = open('/content/ADECal.ics', 'rb')
ecal = Calendar.from_ical(e.read())


i=0
#Calendrier 
for component in ecal.walk():
   if component.name == "VEVENT":
       # Bind prefix to namespace
       g = Graph()
       schema = Namespace("http://schema.org/")
       territoire = Namespace("https://territoire.emse.fr/kg/emse/fayol")
       g.bind('schema', SDO)
       EX = Namespace('https://territoire.emse.fr/ldp/AmineAhmad3/')
       g.bind('ex', EX)
       g.bind('emse',territoire)
       id=str(component.get('UID'))+'/'
       #desc
       desc=str(component.get('DESCRIPTION'))
       audiences=[]
       organizer_1=[]
       for x in desc.split('\n') :
          if 'M1' in x or 'M2' in x :
            audiences.append(x[0:7].strip())
          else :
            organizer_1.append(x.strip())
       organizer_2=[]
       for x in organizer_1 :
         if x!='' and x[0]!='(' :
              organizer_2.append(x.strip())

       #localisation
       localisation =str(component.get('LOCATION')).split(' ')
       for x in localisation :
         if ('1' in x or "2" in x ) and len(x)>=2 and 'EMSE' not in x:
           x=x.replace('A','')
           x=x.replace('S','')
           x=x.replace('.','')
           x=x.replace('L','')
           localisation='/'+x[0]+'ET'+'/'+x
  
       # Reel
       description=component.decoded('SUMMARY').decode('utf-8') 
       #Time,Date
       end =str(component.decoded('DTEND')+timedelta(hours=1)).replace(' ','T')
       start=str(component.decoded('DTSTART')+timedelta(hours=1)).replace(' ','T')
       #start_date=start.split(' ')[0]
       #start_time=start.split(' ')[1]
       #end_time=end.split(' ')[1]
       #graph
       #graph
       if type(localisation) != list:
        event = EX[id]
        g.add((event, RDF.type, schema.Event))
        g.add((event, schema.description, Literal(description, lang='en')))
        g.add((event, schema.course, Literal(True, datatype=XSD.boolean)))
        g.add((event, schema.location,territoire[localisation]))
        g.add((event, schema.location,Literal('Saint-Étienne', lang='fr')))
        g.add((event, schema.startDate, Literal(start, datatype=XSD.dateTime)))
        #g.add((event, schema.startTime, Literal(start_time, datatype=XSD.time)))
        g.add((event, schema.endDate, Literal(end, datatype=XSD.dateTime)))
        for x in organizer_2 :
            g.add((event, schema.organizer, Literal(x)))
        g.add((event, schema.organizer, Literal('EMSE')))
        for x in audiences :
              g.add((event, schema.audience, Literal(x)))
        print(g.serialize(format="ttl").encode())
        '''if i!=0 :
            response = requests.post(url, headers=headers, data=g.serialize(format="ttl").encode())
            print("Status code:", response.status_code)
            if response.status_code == 201:
              location = response.headers.get("Location")
              print("Created resource:", location)
            else:
              print("Error occured :", response.content)
              print(i)
        i+=1'''
        
e.close()

b'@prefix ns1: <http://schema.org/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<https://territoire.emse.fr/ldp/AmineAhmad3/ADE60323032322d3230323353542d455449454e4e452d383032372d342d30/> a ns1:Event ;\n    ns1:audience "M2 CPS2" ;\n    ns1:course true ;\n    ns1:description "CM Multi agent coordination"@en ;\n    ns1:endDate "2022-09-30T11:15:00+00:00"^^xsd:dateTime ;\n    ns1:location <https://territoire.emse.fr/kg/emse/fayol/2ET/221>,\n        "Saint-\xc3\x89tienne"@fr ;\n    ns1:organizer "BALBO FLAVIEN",\n        "EMSE" ;\n    ns1:startDate "2022-09-30T07:00:00+00:00"^^xsd:dateTime .\n\n'
b'@prefix ns1: <http://schema.org/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<https://territoire.emse.fr/ldp/AmineAhmad3/ADE60323032322d3230323353542d455449454e4e452d333039372d31362d30/> a ns1:Event ;\n    ns1:audience "M1 CPS2",\n        "M2 CPS2" ;\n    ns1:course true ;\n    ns1:description "CM/TD Web Programming / Research and development Project"@en ;\n    ns1:en

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**EVENT : FROM THE WEB** (don't use this code)

In [ ]:
from dataclasses import dataclass, asdict
from typing import Dict, Union,List
@dataclass
class Event:
    context: str
    type: str
    id: str
    name: str
    url: str
    description: str
    image: List[str]
    location: Dict[str, Union[str, Dict[str, Union[str,Dict[str, Union[str,str]]]]]]
    startDate: str
    endDate: str

json_ld_list = [{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/2542856","name":"March\u00e9 de Carnot","url":"https://www.alentoor.fr/saint-etienne/agenda/2542856-marche-de-carnot","description":"March\u00e9 de produits manufactur\u00e9s et alimentaires.","image":["https://www.alentoor.fr/assets/img/event/default/marche-medium.jpg"],"location":{"@type":"Place","name":"Place Carnot","address":{"@type":"PostalAddress","streetAddress":"Place Carnot","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-12-31"},{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/2542856","name":"March\u00e9 de Carnot","url":"https://www.alentoor.fr/saint-etienne/agenda/2542856-marche-de-carnot","description":"March\u00e9 de produits manufactur\u00e9s et alimentaires.","image":["https://www.alentoor.fr/assets/img/event/default/marche-medium.jpg"],"location":{"@type":"Place","name":"Place Carnot","address":{"@type":"PostalAddress","streetAddress":"Place Carnot","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-12-31"},\
                {"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/2542857","name":"March\u00e9 de Bellevue","url":"https://www.alentoor.fr/saint-etienne/agenda/2542857-marche-de-bellevue","description":"March\u00e9 de produits manufactur\u00e9s et alimentaires.","image":["https://www.alentoor.fr/assets/img/event/default/marche-medium.jpg"],"location":{"@type":"Place","name":"Place Bellevue","address":{"@type":"PostalAddress","streetAddress":"Place Bellevue","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-12-31"},{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/1502225","name":"March\u00e9 de Montplaisir","url":"https://www.alentoor.fr/saint-etienne/agenda/1502225-marche-de-montplaisir","description":"March\u00e9 de produits manufactur\u00e9s et alimentaires.","image":["https://www.alentoor.fr/photos/classifieds/e0/61/e061354012101c3907e736b604df3246d7edb4a56d26aadcc636ff15d9f9e991-large.jpg"],"location":{"@type":"Place","name":"Rue de terrenoire","address":{"@type":"PostalAddress","streetAddress":"Rue de terrenoire","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-12-29"},\
                {"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3793894","name":"Asse-Laval","url":"https://www.alentoor.fr/saint-etienne/agenda/3793894-asse-laval","description":"Venez nombreux supporter votre \u00e9quipe dans le Chaudron !","image":["https://www.alentoor.fr/photos/classifieds/d7/e7/d7e77888f801cf54b63ff46f64f85167ba3a13d2720143eb2283f89dc1e8e3e7-large.jpg"],"location":{"@type":"Place","name":"Stade Geoffroy-Guichard / Asse","address":{"@type":"PostalAddress","streetAddress":"14 rue Paul et Pierre Guichard","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-01-10"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3211141","name":"Caf\u00e9 Contes au Remue-M\u00e9ninges","url":"https://www.alentoor.fr/saint-etienne/agenda/3211141-cafe-contes-au-remue-meninges","description":"L'automne est l\u00e0, le froid aussi , alors venez vous r\u00e9chauffer avec des histoires , et les th\u00e9s, chocolat chaud mais aussi bi\u00e8res locales du remue meninge pour le caf\u00e9 conte.\n\nTous les 3\u00e8me mardi du m","image":["https://www.alentoor.fr/photos/classifieds/e9/aa/e9aa2643b4c5e3fda0da9a634df5a704937a6c378c204674a947de780e05892c-large.jpg"],"location":{"@type":"Place","name":"Le Remue-M\u00e9ninges","address":{"@type":"PostalAddress","streetAddress":"59 rue D\u00e9sir\u00e9 Claude","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-06-27"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3910886","name":"Exposition \"En guerre d'Alg\u00e9rie. R\u00e9cits partag\u00e9s\"","url":"https://www.alentoor.fr/saint-etienne/agenda/3910886-exposition-en-guerre-d-algerie-recits-partages","description":"L'exposition valorise des t\u00e9moignages oraux ainsi que des objets et documents personnels recueillis aupr\u00e8s d'acteurs encore vivants de l'\u00e9poque, d\u00e9sireux de transmettre leur pass\u00e9 : appel\u00e9s, membres d","image":["https://www.alentoor.fr/photos/classifieds/ef/fc/effc8aa59079b03e8ddadb6e9f79634492724bda5a927433e12d56eb0d3dca17-large.jpg"],"location":{"@type":"Place","name":"Archives municipales et m\u00e9tropolitaines de Saint-Etienne","address":{"@type":"PostalAddress","streetAddress":"164 cours Fauriel","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-05-16"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/4019937","name":"Exposition Hibernation ou le choeur des choses gel\u00e9es","url":"https://www.alentoor.fr/saint-etienne/agenda/4019937-exposition-hibernation-ou-le-choeur-des-choses-gelees","description":"Apr\u00e8s avoir longtemps travaill\u00e9 dans le spectacle vivant, Jean-Jacques Cornillon a fait de la photographie et de l\u2019\u00e9criture ses activit\u00e9s principales. Jusqu\u2019au 24 janvier il investit la galerie R\u00eaves ","image":["https://www.alentoor.fr/photos/classifieds/90/6f/906f956f41a8a0aeed74804109ddd6275b3a15aa539c373bf2e3367b13dcc2a6-large.jpg"],"location":{"@type":"Place","name":"Galerie : R\u00eaves d'Afrique.. et d'Ailleurs","address":{"@type":"PostalAddress","streetAddress":"29 rue Paul  Bert","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-01-24"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3867660","name":"Exposition temporaire :Saint-\u00c9tienne 1780-1856, r/\u00c9volutions","url":"https://www.alentoor.fr/saint-etienne/agenda/3867660-exposition-temporaire-saint-etienne-1780-1856-r-evolutions","description":"Les deux mus\u00e9es de la Ville de Saint-\u00c9tienne, Couriot - Mus\u00e9e de la mine et le mus\u00e9e d\u2019Art et d\u2019Industrie, pr\u00e9sentent deux faces compl\u00e9mentaires pour saisir la premi\u00e8re r\u00e9volution industrielle\n\nDu 22 ","image":["https://www.alentoor.fr/photos/classifieds/08/9c/089c13eefd631f80af1788379b5f26889ecb50e486fe179e23c4fb1bf7c25c7d-large.jpg"],"location":{"@type":"Place","name":"Mus\u00e9e d'Art et d'Industrie","address":{"@type":"PostalAddress","streetAddress":"2 place Louis Comte","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-06-11"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3877533","name":"Jam Session","url":"https://www.alentoor.fr/saint-etienne/agenda/3877533-jam-session","description":"Les \u00e9l\u00e8ves du D\u00e9partement jazz du Conservatoire Massenet proposent \u00e0 tous les musiciens qui le souhaitent de les rejoindre, avec leurs instruments, pour un moment partag\u00e9 de musique.","image":["https://www.alentoor.fr/photos/classifieds/e1/85/e18576b40fb1dc075a4f80b88b25ba65c16422f85c1f9ff849533daf553efb77-large.jpg"],"location":{"@type":"Place","name":"La Com\u00e8te","address":{"@type":"PostalAddress","streetAddress":"7 avenue du Pr\u00e9sident Emile Loubet","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-12-25"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/2542976","name":"March\u00e9 de Saint-Fran\u00e7ois","url":"https://www.alentoor.fr/saint-etienne/agenda/2542976-marche-de-saint-francois","description":"March\u00e9 de produits alimentaires.","image":["https://www.alentoor.fr/photos/classifieds/ca/f4/caf4ab79b3ba1f83626f75b5ba109809b3eb3f518bd13793fdbaa7db2816fa62-large.jpg"],"location":{"@type":"Place","name":"Place Jean Doron","address":{"@type":"PostalAddress","streetAddress":"Place Jean Doron","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-12-31"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3740447","name":"Exposition Bicyclette(s), faire des v\u00e9los","url":"https://www.alentoor.fr/saint-etienne/agenda/3740447-exposition-bicyclette-s-faire-des-velos","description":"Une exposition-\u00e9v\u00e9nement sur le renouveau du v\u00e9lo au XXIe si\u00e8cle avec plus de 70 bicyclettes, v\u00e9ritables bijoux de technologie et de design et la mise en avant des cr\u00e9ateurs, artisans, entreprises et ","image":["https://www.alentoor.fr/photos/classifieds/73/10/7310c0a9ef72922b3674bc295a51aeea53e56f78fad5520318ef160d8e315335-large.jpg"],"location":{"@type":"Place","name":"Cit\u00e9 du Design","address":{"@type":"PostalAddress","streetAddress":"3 rue Javelin Pagnon","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-04-30"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/4013464","name":"Dutronc & Dutronc Annule","url":"https://www.alentoor.fr/saint-etienne/agenda/4013464-dutronc-dutronc-annule","description":"Jacques et Thomas Dutronc se rejoignent sur sc\u00e8ne pour une tourn\u00e9e unique !\nP\u00e8re et fils, monstres sacr\u00e9s de la sc\u00e8ne fran\u00e7aise, vous donnent rendez-vous pour des concerts exceptionnels sous le signe ","image":["https://www.alentoor.fr/photos/classifieds/a5/7c/a57c00a3688cdc707c3e928b6af77438f67852577dd14cecc40d74bb6d153d33-large.jpg"],"location":{"@type":"Place","name":"Z\u00e9nith de Saint-Etienne M\u00e9tropole","address":{"@type":"PostalAddress","streetAddress":"46 rue des Aci\u00e9ries","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-29","endDate":"2023-01-29"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3846979","name":"Le Tresor de Clarissa","url":"https://www.alentoor.fr/saint-etienne/agenda/3846979-le-tresor-de-clarissa","description":"Un voyage qui se transforme en chasse au tr\u00e9sor\u2026    L\u2019aventuri\u00e8re Clarisse Anonyme visite la Louisiane, et apprend l\u2019existence d\u2019une l\u00e9gende\u2026 une l\u00e9gende qui porte son nom ! Ni une, ni deux, munie d\u2019u","image":["https://www.alentoor.fr/photos/classifieds/b1/98/b19878273f4e3606f5c92ab9c1d56b4e915a482dbaa26c2438138d47bb85375c-large.jpg"],"location":{"@type":"Place","name":"4 square Violette","address":{"@type":"PostalAddress","streetAddress":"4 square Violette","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-29","endDate":"2023-01-29"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3846984","name":"P'Tit Loup et le Grand Manitou","url":"https://www.alentoor.fr/saint-etienne/agenda/3846984-p-tit-loup-et-le-grand-manitou","description":"Gratuit pour les moins de 1 an(s) Conte musical, d'apr\u00e8s le conte am\u00e9rindien : le loup qui apprit \u00e0 voler\u00a0       Petit Loup ne veut pas dormir! Il pr\u00e9f\u00e8re \u00e9couter les histoires que lui raconte son gra","image":["https://www.alentoor.fr/photos/classifieds/6b/e0/6be00765516d76f2ed8cc2a5ac3589dc5de27b8463f9704830ad24a396e2f942-large.jpg"],"location":{"@type":"Place","name":"4 square Violette","address":{"@type":"PostalAddress","streetAddress":"4 square Violette","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-29","endDate":"2023-01-29"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3189544","name":"Lucia 3d au Plan\u00e9tarium","url":"https://www.alentoor.fr/saint-etienne/agenda/3189544-lucia-3d-au-planetarium","description":"Diffusion en 3D ! D\u2019o\u00f9 proviennent et comment se produisent les \u00e9toiles filantes ? Quelle est l\u2019origine des m\u00e9t\u00e9orites ?\n\nAlors qu\u2019ils sont en train de contempler une aurore australe depuis l\u2019espace, ","image":["https://www.alentoor.fr/photos/classifieds/0f/c7/0fc76fbb1098a07118788425e1732a878ea4e33d1bea3aa90d83996d4634c07a-large.jpg"],"location":{"@type":"Place","name":"Plan\u00e9tarium de Saint-Etienne (M\u00e9diastro Promotion)","address":{"@type":"PostalAddress","streetAddress":"Cit\u00e9 Fauriel\n28 rue Ponchardier","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-21","endDate":"2023-01-21"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3273123","name":"Entre Terre et Ciel, la Lune au Plan\u00e9tarium","url":"https://www.alentoor.fr/saint-etienne/agenda/3273123-entre-terre-et-ciel-la-lune-au-planetarium","description":"La Lune a \u00e9t\u00e9 observ\u00e9e de tout temps partout sur Terre, elle a permis d\u2019\u00e9tablir les premiers calendriers. Elle a \u00e9t\u00e9 aussi le reflet de nos superstitions et de nos croyances.\n\nD\u00e8s l\u2019antiquit\u00e9 on en co","image":["https://www.alentoor.fr/photos/classifieds/1d/b5/1db5f4f019386e1ca92f2cc72c6874ac784720a1f0d272ff197751f662d39607-large.jpg"],"location":{"@type":"Place","name":"Plan\u00e9tarium de Saint-Etienne","address":{"@type":"PostalAddress","streetAddress":"Espace Fauriel\n28 rue Ponchardier","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-22","endDate":"2023-01-22"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3999380","name":"Atelier Pimp My Bike","url":"https://www.alentoor.fr/saint-etienne/agenda/3999380-atelier-pimp-my-bike","description":"Venez personnaliser vos v\u00e9los avec des couleurs, des rubans des stickers, des pi\u00e8ces de cuir\u2026 \u00e0 votre disposition !","image":["https://www.alentoor.fr/photos/classifieds/7e/87/7e8732226ad3560636ac4cd3e6aff40d631ca184600ba7cdc739577f7cfb9331-large.jpg"],"location":{"@type":"Place","name":"Cit\u00e9 du Design","address":{"@type":"PostalAddress","streetAddress":"3 rue Javelin Pagnon","addressLocality":"Saint-\u00c9tienne","postalCode":"42000","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-04-01"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3960223","name":"Exposition - The House Of Dust - au Mamc","url":"https://www.alentoor.fr/saint-priest-en-jarez/agenda/3960223-exposition-the-house-of-dust-au-mamc","description":"En \u00e9cho \u00e0 sa programmation attentive aux artistes femmes, le MAMC+ propose un nouvel accrochage des collections s\u2019attachant \u00e0 am\u00e9liorer la visibilit\u00e9 de la cr\u00e9ation f\u00e9minine, qui ne repr\u00e9sente qu\u2019envi","image":["https://www.alentoor.fr/photos/classifieds/c9/52/c9524b71ae35e735d8ab426d9fcc26836287d829fdaae594046bfeb5e9415e5c-large.jpg"],"location":{"@type":"Place","name":"Mus\u00e9e d'art moderne et contemporain Saint-Etienne M\u00e9tropole","address":{"@type":"PostalAddress","streetAddress":"Rue Fernand L\u00e9ger","addressLocality":"Saint-Priest-en-Jarez","postalCode":"42270","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-04-10"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/4013502","name":"Spectacle - les (pas tant) petits caraoquets (de conserve)","url":"https://www.alentoor.fr/la-ricamarie/agenda/4013502-spectacle-les-pas-tant-petits-caraoquets-de-conserve","description":"Un karaok\u00e9 th\u00e9\u00e2tral fait maison avec les moyens du bord : un spectacle participatif, joyeux et bord\u00e9lique pour rire et chanter \u00e0 gorge d\u00e9ploy\u00e9e !\n\nLes Gaillard, c\u02bcest une histoire de famille. Chaque m","image":["https://www.alentoor.fr/photos/classifieds/29/00/2900d3b8c3273e9e1ec337572b28e104a0e51ad98f3449c5254c1b41ad8f5a2b-large.jpg"],"location":{"@type":"Place","name":"Centre Culturel de la Ricamarie","address":{"@type":"PostalAddress","streetAddress":"Salle Louis Daquin\nAvenue Maurice Thorez","addressLocality":"La Ricamarie","postalCode":"42150","addressCountry":"FR"}},"startDate":"2023-01-29","endDate":"2023-01-29"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3888921","name":"La promenade de Pinceau","url":"https://www.alentoor.fr/firminy/agenda/3888921-la-promenade-de-pinceau","description":"Un livret d\u00e9couverte tr\u00e8s ludique, parfait pour les parents qui souhaitent faire d\u00e9couvrir l'architecture \u00e0 leur enfant de 5 \u00e0 9 ans.\nGr\u00e2ce \u00e0 la balade de pinceau vous partagerez un beau moment de com","image":["https://www.alentoor.fr/photos/classifieds/4d/04/4d0477dfc3d8d615d16e085af551fd55f392416de49914b46ffd4b63b707c7fa-large.jpg"],"location":{"@type":"Place","name":"Site le Corbusier","address":{"@type":"PostalAddress","streetAddress":"Boulevard p\u00e9riph\u00e9rique du Stade","addressLocality":"Firminy","postalCode":"42700","addressCountry":"FR"}},"startDate":"2023-01-10","endDate":"2023-02-05"},\
	{"@context":"http://schema.org","@type":"Event","@id":"https://www.alentoor.fr/agenda/3999309","name":"Un Dimanche en Famille \u00ab l\u2019architecture dans tous les sens \u00bb","url":"https://www.alentoor.fr/firminy/agenda/3999309-un-dimanche-en-famille-l-architecture-dans-tous-les-sens","description":"\u00ab L\u2019architecture dans tous les sens \u00bb Mettez tous vos sens en \u00e9veil !\n\nDimanches 15 et 29 janvier \u00e0 15h\n\nMettez tous vos sens en \u00e9veil !\nVibrez au contact de la mati\u00e8re, r\u00e9sonnez de l\u2019acoustique des l","image":["https://www.alentoor.fr/photos/classifieds/46/6c/466c72e4ca2d2a0ed07262a048c2fd4fe913dbac36e1d5bd48c44cb58f53d613-large.jpg"],"location":{"@type":"Place","name":"Site le Corbusier","address":{"@type":"PostalAddress","streetAddress":"Boulevard p\u00e9riph\u00e9rique du Stade","addressLocality":"Firminy","postalCode":"42700","addressCountry":"FR"}},"startDate":"2023-01-15","endDate":"2023-01-15"}]
id=1
for x in json_ld_list :
    g = Graph()
    schema = Namespace("http://schema.org/")
    territoire = Namespace("https://territoire.emse.fr/kg/emse/fayol")
    g.bind('schema', SDO)
    EX = Namespace('https://territoire.emse.fr/ldp/AmineAhmad3/')
    g.bind('ex', EX)
    g.bind('emse',territoire)
    x = {k.lstrip("@"): v for k, v in x.items()}
    event = Event(**x)
    id_name='ID'+str(id) +'/'
    event_ = EX[id_name]
    startdate=str(event.startDate)+'T08:00:00+00:00'
    enddate=str(event.endDate)+'T16:00:00+00:00'
    g.add((event_, RDF.type, schema.Event))
    g.add((event_, schema.description, Literal(str(event.name), lang='fr')))
    g.add((event_, schema.course, Literal(False, datatype=XSD.boolean)))
    g.add((event_, schema.location,Literal(event.location['address']['streetAddress'], lang='fr')))
    g.add((event_, schema.location,Literal('Saint-Étienne', lang='fr')))
    g.add((event_, schema.startDate, Literal(startdate, datatype=XSD.dateTime)))
    g.add((event_, schema.endDate, Literal(enddate, datatype=XSD.dateTime)))
    response = requests.post(url, headers=headers, data=g.serialize(format="ttl").encode())
    print("Status code:", response.status_code)
    if response.status_code == 201:
      location = response.headers.get("Location")
      print("Created resource:", location)
    else:
      print("Error occured :", response.content)
      print(id)
    id+=1

    

Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID1/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID2/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID3/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID4/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID5/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID6/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID7/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID8/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID9/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID10/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID11/
Status code: 201
Created resource: https://territoire.emse.fr/ldp/AmineAhmad3/ID12/
S

**Requette :**

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
def event_come_after_a_date(date) :
    sparql = SPARQLWrapper("https://territoire.emse.fr/ldp/AmineAhmad3/")
    sparql.setCredentials('ldpuser','LinkedDataIsGreat')
    # Set the query string
    #date='2023-01-10T00:00:00'
    sparql.setQuery('''
        PREFIX schema: <http://schema.org/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT ?event ?description ?startDate ?endDate  ?location1 ?location2
        WHERE {
            ?event a schema:Event .
            ?event schema:description ?description .
            ?event schema:startDate ?startDate .
            ?event schema:endDate ?endDate .
            ?event schema:location ?location1.
            ?event schema:location ?location2.
            <https://territoire.emse.fr/ldp/AmineAhmad3/> <http://www.w3.org/ns/ldp#member> ?event .
            FILTER (?startDate >= "'''+date+'''"^^xsd:dateTime) .
            FILTER (?location1 != ?location2) .
            }
    ''')

    # Set the returned format as JSON
    sparql.setReturnFormat(JSON)
    # Execute the query and get the results
    results = sparql.query().convert()
    #print(results["results"]["bindings"])
    # Print the results
    i=0 
    for result in results["results"]["bindings"]:
        if  i==0: 
          print(result["event"]["value"].split('/')[-2], result["description"]["value"],result["startDate"]["value"], result["endDate"]["value"],result["location1"]["value"],result["location2"]["value"])
          i=1
        else : i=0
        

event_come_after_a_date("amine")

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

def today_event() :
    sparql = SPARQLWrapper("https://territoire.emse.fr/ldp/AmineAhmad3/")
    sparql.setCredentials('ldpuser','LinkedDataIsGreat')
    # Set the query string
    sparql.setQuery('''
        PREFIX schema: <http://schema.org/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT ?event ?description ?startDate ?endDate ?location1 ?location2
        WHERE {
            ?event a schema:Event .
            ?event schema:description ?description .
            ?event schema:startDate ?startDate .
            ?event schema:endDate ?endDate .
            ?event schema:location ?location1 .
            ?event schema:location ?location2 .
            <https://territoire.emse.fr/ldp/AmineAhmad3/> <http://www.w3.org/ns/ldp#member> ?event .
            FILTER((?startDate >= "TODAYT00:00:00"^^xsd:dateTime && ?endDate <= "TODAYT23:59:59"^^xsd:dateTime) || 
            (?startDate <= NOW() && ?endDate >= NOW())).
            FILTER (?location1 != ?location2)} 
    ''')

    # Set the returned format as JSON
    sparql.setReturnFormat(JSON)
    # Execute the query and get the results
    results = sparql.query().convert()
    #print(results["results"]["bindings"])
    # Print the results
    i=0 
    for result in results["results"]["bindings"]:
        if  i==0: 
          print(result["event"]["value"].split('/')[-2], result["description"]["value"],result["startDate"]["value"], result["endDate"]["value"],result["location1"]["value"],result["location2"]["value"])
          i=1
        else : i=0
today_event() 

ID1 Marché de Carnot 2023-01-10T08:00:00+00:00 2023-12-31T16:00:00+00:00 Place Carnot Saint-Étienne
ID2 Marché de Carnot 2023-01-10T08:00:00+00:00 2023-12-31T16:00:00+00:00 Place Carnot Saint-Étienne
ID3 Marché de Bellevue 2023-01-10T08:00:00+00:00 2023-12-31T16:00:00+00:00 Saint-Étienne Place Bellevue
ID4 Marché de Montplaisir 2023-01-10T08:00:00+00:00 2023-12-29T16:00:00+00:00 Saint-Étienne Rue de terrenoire
ID6 Café Contes au Remue-Méninges 2023-01-10T08:00:00+00:00 2023-06-27T16:00:00+00:00 Saint-Étienne 59 rue Désiré Claude
ID7 Exposition "En guerre d'Algérie. Récits partagés" 2023-01-10T08:00:00+00:00 2023-05-16T16:00:00+00:00 Saint-Étienne 164 cours Fauriel
ID8 Exposition Hibernation ou le choeur des choses gelées 2023-01-10T08:00:00+00:00 2023-01-24T16:00:00+00:00 Saint-Étienne 29 rue Paul  Bert
ID9 Exposition temporaire :Saint-Étienne 1780-1856, r/Évolutions 2023-01-10T08:00:00+00:00 2023-06-11T16:00:00+00:00 Saint-Étienne 2 place Louis Comte
ID10 Jam Session 2023-01-10T08:00:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

def today_event_after_now() :
    sparql = SPARQLWrapper("https://territoire.emse.fr/ldp/AmineAhmad3/")
    sparql.setCredentials('ldpuser','LinkedDataIsGreat')
    # Set the query string
    sparql.setQuery('''
        PREFIX schema: <http://schema.org/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT ?event ?name ?startDate ?location1 ?location2
        WHERE {
            ?event a schema:Event .
            ?event schema:description ?description .
            ?event schema:startDate ?startDate .
            ?event schema:endDate ?endDate .
            ?event schema:location ?location1 .
            ?event schema:location ?location2 .
            <https://territoire.emse.fr/ldp/AmineAhmad3/> <http://www.w3.org/ns/ldp#member> ?event .
            FILTER(?endDate <= "TODAYT23:59:59"^^xsd:dateTime && ?startDate >= now() ).
            FILTER (?location1 != ?location2)}
    ''')

    # Set the returned format as JSON
    sparql.setReturnFormat(JSON)
    # Execute the query and get the results
    results = sparql.query().convert()
    #print(results["results"]["bindings"])
    # Print the results
    i=0 
    for result in results["results"]["bindings"]:
        if  i==0: 
          print(result["event"]["value"].split('/')[-2], result["description"]["value"],result["startDate"]["value"], result["endDate"]["value"],result["location1"]["value"],result["location2"]["value"])
          i=1
        else : i=0
today_event_after_now()

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

def today_course(bool_t) :
  sparql = SPARQLWrapper("https://territoire.emse.fr/ldp/AmineAhmad3/")
  sparql.setCredentials('ldpuser','LinkedDataIsGreat')
  # Set the query string

  sparql.setQuery('''
      PREFIX schema: <http://schema.org/>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
     SELECT ?event ?description ?startDate ?endDate ?location1 ?location2
        WHERE {
          ?event a schema:Event .
          ?event schema:description ?description .
          ?event schema:startDate ?startDate .
          ?event schema:endDate ?endDate .
          ?event schema:location ?location1 .
          ?event schema:location ?location2 .
          ?event schema:course '''+str(bool_t)+'''.
          <https://territoire.emse.fr/ldp/AmineAhmad3/> <http://www.w3.org/ns/ldp#member> ?event .
          FILTER((?startDate >= "TODAYT00:00:00"^^xsd:dateTime && ?endDate <= "TODAYT23:59:59"^^xsd:dateTime)
           || 
            (?startDate <= NOW() && ?endDate >= NOW())).
          FILTER (?location1 != ?location2)}
  ''')

  # Set the returned format as JSON
  sparql.setReturnFormat(JSON)
  # Execute the query and get the results
  results = sparql.query().convert()
  #print(results["results"]["bindings"])
  # Print the results
  i=0 
  for result in results["results"]["bindings"]:
        if  i==0: 
          print(result["event"]["value"].split('/')[-2], result["description"]["value"],result["startDate"]["value"], result["endDate"]["value"],result["location1"]["value"],result["location2"]["value"])
          i=1
        else : i=0
today_course(True)

In [ ]:

from SPARQLWrapper import SPARQLWrapper, JSON
 
def ST_event() :
  sparql = SPARQLWrapper("https://territoire.emse.fr/ldp/AmineAhmad3/")
  sparql.setCredentials('ldpuser','LinkedDataIsGreat')
  # Set the query string

  sparql.setQuery('''
      PREFIX schema: <http://schema.org/>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?event ?description ?startDate ?endDate  ?location
      WHERE {
          ?event a schema:Event ;
                schema:location "Saint-Étienne"@fr;
                schema:location ?location.
          ?event schema:description ?description .
          ?event schema:startDate ?startDate .
          ?event schema:endDate ?endDate .
          ?event schema:startDate ?startDate .
          <https://territoire.emse.fr/ldp/AmineAhmad3/> <http://www.w3.org/ns/ldp#member> ?event .}
  ''')

  # Set the returned format as JSON
  sparql.setReturnFormat(JSON)
  # Execute the query and get the results
  results = sparql.query().convert()
  #print(results["results"]["bindings"])
  # Print the results
  for result in results["results"]["bindings"]:
        if  result["location"]["value"]!= "Saint-Étienne": 
          print(result["event"]["value"].split('/')[-2], result["description"]["value"],result["startDate"]["value"], result["endDate"]["value"],result["location"]["value"])
    

Modification :

In [ ]:
import subprocess
def modification(resource_uri,name) :
      # Set the URI of the resource you want to edit
      
      url = "https://territoire.emse.fr/ldp/AmineAhmad3/"
      slug = 'AmineAhmad'
      username = 'ldpuser'
      password = 'LinkedDataIsGreat'
      credentials = f'{username}:{password}'.encode('utf-8')
      encoded_credentials = b64encode(credentials).decode('utf-8')

      headers = {
              'Content-Type': 'text/turtle',
              'Authorization': f'Basic {encoded_credentials}',
              'Slug': slug,
          }
      # Retrieve the resource's turtle representation
      curl_command = f'curl -u ldpuser:LinkedDataIsGreat -X GET {resource_uri} -H "Accept: text/turtle"'
      response = subprocess.run(curl_command, shell=True, capture_output=True)

      g = Graph()
      turtle_data = response.stdout.decode("utf-8")

      list_=[turtle_data.split(';')[0].split(',')[0]] + turtle_data.split(';')[1:-5]
      
      list_=';'.join(list_) +';' + '\n<http://schema.org/attendee> "'+name+ '"@fr .'
      

      g.parse(data=list_, format='ttl')

      # Retrieve the resource's turtle representation
      curl_command = f'curl -u ldpuser:LinkedDataIsGreat -X DELETE {resource_uri} '
      response = subprocess.run(curl_command, shell=True, capture_output=True)

      resource_uri="https://territoire.emse.fr/ldp/AmineAhmad3/"
      response = requests.post(resource_uri, headers=headers, data=g.serialize(format="ttl").encode())
      #print("Status code:", response.status_code)
      if response.status_code == 201:
        #location = response.headers.get("Location")
        #print("Created resource:", location)
        print('Done')
      else:
        print("I can't do it !")



**Validation :**

In [ ]:
pip install pyshacl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.5 MB/s eta 0:00:00
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.6.0
    Uninstalling prettytable-3.6.0:
      Successfully uninstalled prettytable-3.6.0
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1


In [ ]:
from pyshacl import validate
from rdflib import Graph
import pyshacl
def its_event(event) :
    # Create an RDF graph to store the data
    g=Graph()
    schema = Namespace("http://schema.org/")
    territoire = Namespace("https://territoire.emse.fr/kg/emse/fayol")
    g.bind('schema', SDO)
    EX = Namespace('https://territoire.emse.fr/ldp/AmineAhmad3/')
    g.bind('ex', EX)
    g.bind('emse',territoire)

    id_name='ID'+str(event['id']) +'/'
    event_ = EX[id_name]
    startdate=str(event['startDate'])+'T'+str(event['startH'])
    enddate=str(event['endDate'])+'T'+str(event['endH'])
    g.add((event_, RDF.type, schema.Event))
    if event['description']!='':
         g.add((event_, schema.description, Literal(str(event['description']), lang='fr')))
    g.add((event_, schema.course, Literal(False, datatype=XSD.boolean)))
    if event['location1']!='' :
       g.add((event_, schema.location,Literal(event['location1'])))
    if event['location2']!='' :
       g.add((event_, schema.location,Literal(event['location2'])))
    g.add((event_, schema.startDate, Literal(startdate, datatype=XSD.dateTime)))
    g.add((event_, schema.endDate, Literal(enddate, datatype=XSD.dateTime)))
    #print(g.serialize(format='turtle'))
    # Create an RDF graph to store the shapes
    with open("/content/event.txt", "r") as f:
        ttl_str_shape = f.read()

    shapes = Graph().parse(data=ttl_str_shape, format="ttl")
    conforms,m,d= validate(g, shacl_graph=shapes)

    if conforms:
        print("The data conforms to the shapes.")
        url = "https://territoire.emse.fr/ldp/AmineAhmad3/"
        #slug = 'AmineAhmad'
        username = 'ldpuser'
        password = 'LinkedDataIsGreat'
        credentials = f'{username}:{password}'.encode('utf-8')
        encoded_credentials = b64encode(credentials).decode('utf-8')

        headers = {
                'Content-Type': 'text/turtle',
                'Authorization': f'Basic {encoded_credentials}',
              
            }
        response = requests.post(url, headers=headers, data=g.serialize(format="ttl").encode())

    else:
        print("The data does not conform to the shapes.")

In [ ]:
from pyshacl import validate
from rdflib import Graph
def its_course(event) :
    # Create an RDF graph to store the data
    g=Graph()
    schema = Namespace("http://schema.org/")
    territoire = Namespace("https://territoire.emse.fr/kg/emse/fayol")
    g.bind('schema', SDO)
    EX = Namespace('https://territoire.emse.fr/ldp/AmineAhmad3/')
    g.bind('ex', EX)
    g.bind('emse',territoire)

    id_name=str(event['id']) +'/'
    event_ = EX[id_name]
    startdate=str(event['startDate'])+'T'+str(event['startH'])
    enddate=str(event['startDate'])+'T'+str(event['endH'])
    g.add((event_, RDF.type, schema.Event))
    if event['description']!='':
         g.add((event_, schema.description, Literal(str(event['description']), lang='fr')))
    g.add((event_, schema.course, Literal(False, datatype=XSD.boolean)))
    if event['location']!='' :
      if event['location'][0]=='R' :
         g.add((event_, schema.location,Literal(event['location'])))
         #g.add((event_, schema.location,Literal('UJM')))
      else :
        g.add((event_, schema.location,territoire[event['location']]))
    g.add((event_, schema.location,Literal('Saint-Étienne')))
    if startdate!='T' :
        g.add((event_, schema.startDate, Literal(startdate, datatype=XSD.dateTime)))
    if enddate!='T' :
      g.add((event_, schema.endDate, Literal(enddate, datatype=XSD.dateTime)))
    for x in event['organized'] :
      if x!='' :
        g.add((event_, schema.organized, Literal(x)))
    print(g.serialize(format='turtle'))
    # Create an RDF graph to store the shapes
    with open("/content/course.txt", "r") as f:
        ttl_str_shape = f.read()

    shapes = Graph().parse(data=ttl_str_shape, format="ttl")


    # Perform the validation
    conforms, v_graph, v_text = validate(g, shacl_graph=shapes)

    # Print the validation results
    if conforms:
        print("The data conforms to the shapes.")
        url = "https://territoire.emse.fr/ldp/AmineAhmad3/"
        #slug = 'AmineAhmad'
        username = 'ldpuser'
        password = 'LinkedDataIsGreat'
        credentials = f'{username}:{password}'.encode('utf-8')
        encoded_credentials = b64encode(credentials).decode('utf-8')

        headers = {
                'Content-Type': 'text/turtle',
                'Authorization': f'Basic {encoded_credentials}',
                
            }
        response = requests.post(url, headers=headers, data=g.serialize(format="ttl").encode())

    else:
        print("The data does not conform to the shapes.")

**Menu :**

In [ ]:
from IPython.display import clear_output
# init the calendar
from datetime import datetime, timedelta
from rdflib import Graph
import requests 
from base64 import b64encode
from SPARQLWrapper import SPARQLWrapper, JSON
import subprocess
from pyshacl import validate
from rdflib import Graph
def display_menu():
    clear_output()
    print("1. List of events after this date")
    print("2. Today events")
    print("3. Today events that didn't started yet ")
    print("4. Today courses ")
    print("5. Today events that not a course ")
    print("6. List of events in Saint Etienne ")
    print("7. Register your attendance")
    print("8. Add an event ")
    print("9. Add a course ")
    print("10. Exit")

while True:
    display_menu()
    choice = input("Enter your choice: ")
    
    if choice == "1":
        date=input("Enter the date...")
        hour=input("Enter the hour if you want (like 00:00:00)...")
        if hour=='' : 
          date+='T00:00:00'
        else :
          date+='T'+hour
        event_come_after_a_date(date)
    elif choice == "2":
        today_event()
    elif choice == "3":
        today_event_after_now()
    elif choice == "4":
        today_course(True)
    elif choice == "5":
        today_course(False)
    elif choice == "6":
        ST_event()
    elif choice == "7":
        id=input("Enter the event id...")
        name=input("Enter your name")
        if id!='' and name!='' :
          id='https://territoire.emse.fr/ldp/AmineAhmad3/'+id+'/'
          modification(id,name)
        else :
          print("Invalid choice. Please try again.")
    elif choice == "8":
        event={}
        event['id']=input("Enter the id...")
        event['description']=input("Enter the event name...")
        event['startDate']=input("Enter the event start Date (YYYY-MM-DD)...") 
        event['startH']=input("Enter the event start hour (HH:MM:SS)...")
        event['endDate']=input("Enter the event end Date (YYYY-MM-DD)...")
        event['endH']=input("Enter the event end hour (HH:MM:SS)...")
        event['location1']=input("Enter the event location...")
        event['location2']=input("Enter the event city (Full name)...")
        if event['startDate']!='' and event['startH']!='' and event['endDate']!='' and event['endH']!='': its_event(event)
        else : print("Invalid choice. Please try again.")
    elif choice == "9":
        event={}
        event['id']=input("Enter the id...")
        event['description']=input("Enter the event name...")
        event['startDate']=input("Enter the event start Date (YYYY-MM-DD)...") 
        event['startH']=input("Enter the event start hour (HH:MM:SS)...")
        #event['endDate']=input("Enter the event end Date (YYYY-MM-DD)...")
        event['endH']=input("Enter the event end hour (HH:MM:SS)...")
        event['organized']=[input("Enter the event Prof...")]
        event['organized']+=[input("Enter the event Prof 2 (if exists)...")]
        event['organized']+=[input("Enter the event school (if exists)...")]
        loc=input("Enter the room number (3 digits)...")
        if event['organized'][-1]=='EMSE' :
          event['location']='/'+loc[0]+'ET'+'/'+loc
        else :
          event['location']='Room '+loc+' ,UJM'
        if event['startDate']!='' and event['startH']!=''  and event['endH']!='': its_course(event)
        else : print("Invalid choice. Please try again.")
        its_course(event)
    elif choice == "10":
        break
    else:
        print("Invalid choice. Please try again.")
    input("Press enter to continue...")

1. List of events after this date
2. Today events
3. Today events that didn't started yet 
4. Today courses 
5. Today events that not a course 
6. List of events in Saint Etienne 
7. Register your attendance
8. Add an event 
9. Add a course 
10. Exit
Enter your choice: 9
Enter the id...ID1555
Enter the event name...a
Enter the event start Date (YYYY-MM-DD)...2023-01-13
Enter the event start hour (HH:MM:SS)...14:00:00
Enter the event end hour (HH:MM:SS)...16:00:00
Enter the event Prof...Amine
Enter the event Prof 2 (if exists)...
Enter the event school (if exists)...EMSE
Enter the room number (3 digits)...123
@prefix ns1: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://territoire.emse.fr/ldp/AmineAhmad3/ID1555/> a ns1:Event ;
    ns1:course false ;
    ns1:description "a"@fr ;
    ns1:endDate "2023-01-13T16:00:00"^^xsd:dateTime ;
    ns1:location <https://territoire.emse.fr/kg/emse/fayol/1ET/123>,
        "Saint-Étienne" ;
    ns1:organized "Amine",
 

In [ ]:
import subprocess
resource_uri = "https://territoire.emse.fr/ldp/AmineAhmad3/ID27/"

# Retrieve the resource's turtle representation
curl_command = f'curl -u ldpuser:LinkedDataIsGreat -X DELETE {resource_uri} '
response = subprocess.run(curl_command, shell=True, capture_output=True)